In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
from pmdarima import auto_arima
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import statsmodels.api as sm
import matplotlib.dates as mdates
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings("ignore")
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', UserWarning)
warnings.simplefilter('ignore', FutureWarning)
warnings.simplefilter('ignore', ConvergenceWarning)



In [8]:
import yfinance as yf

ticker = 'ETH-USD'
start_date = '2017-01-01'
end_date = '2025-04-13'

eth_data = yf.download(ticker, start=start_date, end=end_date, interval='1d')
eth_data.reset_index(inplace=True)

eth_data.to_csv('eth_usd_2017_2025.csv', index=False)


[*********************100%***********************]  1 of 1 completed


In [10]:
eth_data.shape

(2712, 6)

In [11]:
eth_data.head()

Price,Date,Close,High,Low,Open,Volume
Ticker,,ETH-USD,ETH-USD,ETH-USD,ETH-USD,ETH-USD
0,2017-11-09,320.884003,329.451996,307.056000,308.644989,893249984
1,2017-11-10,299.252991,324.717987,294.541992,320.670990,885985984
2,2017-11-11,314.681000,319.453003,298.191986,298.585999,842300992
3,2017-11-12,307.907990,319.153015,298.513000,314.690002,1613479936
4,2017-11-13,316.716003,328.415009,307.024994,307.024994,1041889984


In [14]:
eth_data.tail()

Price,Date,Close,High,Low,Open,Volume
Ticker,,ETH-USD,ETH-USD,ETH-USD,ETH-USD,ETH-USD
2707,2025-04-08,1472.553101,1617.339844,1447.610107,1554.932129,21315312919
2708,2025-04-09,1668.040039,1687.187988,1386.799316,1472.601440,39252195855
2709,2025-04-10,1522.518799,1669.394165,1474.914185,1668.201782,21379604307
2710,2025-04-11,1567.149780,1587.538574,1505.001953,1522.473022,14871838813
2711,2025-04-12,1643.528564,1666.017578,1546.819580,1567.155762,12110995013


In [12]:
eth_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2712 entries, 0 to 2711
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   (Date, )           2712 non-null   datetime64[ns]
 1   (Close, ETH-USD)   2712 non-null   float64       
 2   (High, ETH-USD)    2712 non-null   float64       
 3   (Low, ETH-USD)     2712 non-null   float64       
 4   (Open, ETH-USD)    2712 non-null   float64       
 5   (Volume, ETH-USD)  2712 non-null   int64         
dtypes: datetime64[ns](1), float64(4), int64(1)
memory usage: 127.3 KB


In [ ]:
eth_data.describe()
eth_data['Date'] = pd.to_datetime(eth_data['Date'])

Price,Date,Close,High,Low,Open,Volume
Ticker,,ETH-USD,ETH-USD,ETH-USD,ETH-USD,ETH-USD
count,2712,2712.000000,2712.000000,2712.000000,2712.000000,2.712000e+03
mean,2021-07-26 12:00:00,1531.461546,1572.993085,1484.548041,1531.133486,1.340000e+10
min,2017-11-09 00:00:00,84.308296,85.342743,82.829887,84.279694,6.217330e+08
25%,2019-09-17 18:00:00,283.829758,291.495743,273.807487,283.963997,5.800033e+09
50%,2021-07-26 12:00:00,1540.920532,1578.116394,1494.774597,1539.604431,1.093713e+10
75%,2023-06-04 06:00:00,2467.183350,2544.766479,2382.881287,2467.303223,1.811068e+10
max,2025-04-12 00:00:00,4812.087402,4891.704590,4718.039062,4810.071289,9.245355e+10
std,NaN,1226.133041,1259.103096,1189.825787,1226.614252,1.083710e+10


In [15]:
eth_data.columns = [col[0] if isinstance(col, tuple) else col for col in eth_data.columns] # Converted MultiIndex to a single index for improved data accessibility and readability <3
eth_data.head()

,Date,Close,High,Low,Open,Volume
0,2017-11-09,320.884003,329.451996,307.056000,308.644989,893249984
1,2017-11-10,299.252991,324.717987,294.541992,320.670990,885985984
2,2017-11-11,314.681000,319.453003,298.191986,298.585999,842300992
3,2017-11-12,307.907990,319.153015,298.513000,314.690002,1613479936
4,2017-11-13,316.716003,328.415009,307.024994,307.024994,1041889984
